# **Mount Google Drive**

In [ ]:
from google.colab import drive
import sys

drive.mount('/content/drive')


# **Import Neccesary Packages**

In [ ]:
!pip install -q transformers datasets scikit-learn seaborn


import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import CLIPProcessor, CLIPTokenizer, CLIPModel, get_scheduler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm.notebook import tqdm
import glob
import gc
import matplotlib.pyplot as plt
import seaborn as sns

# **Function for Configuring Files and Hyperparamters**

In [ ]:
class CFG_C:
    """
    Configuration Class for Subtask C: Stance Detection.
    """
    DRIVE_PATH = "/content/drive/MyDrive/case-2025"

    # File & Directory Paths
    train_dir = os.path.join(DRIVE_PATH, "SubTaskC/Train")
    train_text_path = os.path.join(DRIVE_PATH, "SubTaskC/Train/STask_C_train.csv")
    val_image_dir = os.path.join(DRIVE_PATH, "SubTaskC/Eval/STask_C_val_img")
    val_labels_path = os.path.join(DRIVE_PATH, "SubTaskC/Eval/STask-C(index,label)val.csv")
    val_text_path = os.path.join(DRIVE_PATH, "SubTaskC/Eval/STask-C(index,text)val.csv")
    test_image_dir = os.path.join(DRIVE_PATH, "SubTaskC/Test/STask_C_test_img")
    test_csv_path = os.path.join(DRIVE_PATH, "SubTaskC/Test/STask-C(index,text)test.csv")
    output_dir = os.path.join(DRIVE_PATH, "output_subtask_c")

    # Model & Training Parameters
    model_name = 'openai/clip-vit-large-patch14'
    image_size = 224
    max_token_len = 77
    learning_rate_base = 1e-6
    learning_rate_head = 1e-5
    batch_size = 16
    epochs = 10
    num_workers = 2
    device = "cuda" if torch.cuda.is_available() else "cpu"

os.makedirs(CFG_C.output_dir, exist_ok=True)
print(f"Subtask C Configuration defined. Output will be saved to: {CFG_C.output_dir}")

# Data Loading & Reconnaissance
print("\nLoading Subtask C data...")
try:
    #Training Data Loading
    train_image_paths = glob.glob(os.path.join(CFG_C.train_dir, '**/*.png'), recursive=True)
    train_data = []
    class_folders = ['Neutral', 'Support', 'Oppose']
    for path in train_image_paths:
        label = os.path.basename(os.path.dirname(path))
        if label in class_folders:
            train_data.append({'index': os.path.basename(path), 'label_text': label})

    ground_truth_labels_df = pd.DataFrame(train_data)
    text_data_df = pd.read_csv(CFG_C.train_text_path, usecols=['index', 'text'])
    train_df_c = pd.merge(ground_truth_labels_df, text_data_df, on="index")

    #Validation Data Loading
    val_labels_df = pd.read_csv(CFG_C.val_labels_path)
    val_text_df = pd.read_csv(CFG_C.val_text_path)
    val_df_c = pd.merge(val_text_df, val_labels_df, on="index")

    print("Data loaded successfully.")
    print(f"Training samples:   {len(train_df_c)}")
    print(f"Validation samples: {len(val_df_c)}")

except FileNotFoundError as e:
    print(f"\nFATAL ERROR: A data file was not found. Please double-check your paths in CFG_C.")
    print(f"Details: {e}")
    sys.exit()

# Visualization: Label Distribution

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(10, 6))
class_order = ['Neutral', 'Support', 'Oppose']
sns.countplot(x='label_text', data=train_df_c, palette='crest', order=class_order)
plt.title('Subtask C: Training Set Label Distribution')
plt.xlabel('Stance Class')
plt.ylabel('Count')
plt.show()

# **DATA PREPARATION**

In [ ]:
print("Preparing Subtask C data: Enforcing official label encoding.")
official_target_map_c = {
    'Neutral': 0,
    'Support': 1,
    'Oppose': 2
}
print(f"   - Official Target Map: {official_target_map_c}")

# Create the 'label_encoded' column for the training set from the 'label_text' column.
train_df_c['label_encoded'] = train_df_c['label_text'].map(official_target_map_c)

val_df_c.rename(columns={'label': 'label_encoded'}, inplace=True)
val_df_c['label_encoded'] = val_df_c['label_encoded'].astype(int)

if train_df_c['label_encoded'].isnull().any() or val_df_c['label_encoded'].isnull().any():
    print(" WARNING: Null values found after processing. Please investigate.")
else:
    print("Label encoding complete and columns aligned.")

CFG_C.num_classes = len(official_target_map_c)
print(f"   - Number of classes for Subtask C: {CFG_C.num_classes}")

# **BUILDING THE DATA PIPELINE**

In [ ]:
#Load the Official CLIP Processor
processor = CLIPProcessor.from_pretrained(CFG_C.model_name)

# Define the  Dataset Class
class StanceDataset(Dataset):
    def __init__(self, df, processor, image_dir, image_size, max_token_len, is_test=False):
        self.df = df
        self.processor = processor
        self.image_dir = image_dir
        self.image_size = image_size
        self.max_token_len = max_token_len
        self.is_test = is_test
        self.image_path_map = {os.path.basename(p): p for p in glob.glob(os.path.join(image_dir, '**/*.*'), recursive=True)}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = str(row.get('text', ''))

        tokenized = self.processor(text=text, truncation=True, max_length=self.max_token_len, padding="max_length", return_tensors="pt")
        image_name = row['index']
        image_path = self.image_path_map.get(image_name)
        if image_path:
            image = Image.open(image_path).convert("RGB")
        else:
            image = Image.new('RGB', (self.image_size, self.image_size), 'black')
        processed_image = self.processor(images=image, return_tensors="pt")

        # Assemble the item
        item = {
            'input_ids': tokenized['input_ids'].squeeze(0),
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'pixel_values': processed_image['pixel_values'].squeeze(0)
        }
        if not self.is_test:
            item['label'] = torch.tensor(row['label_encoded'], dtype=torch.long)
        return item

# DataLoaders for Subtask C
test_df_c = pd.read_csv(CFG_C.test_csv_path)

train_dataset_c = StanceDataset(train_df_c, processor, CFG_C.train_dir, CFG_C.image_size, CFG_C.max_token_len)
val_dataset_c = StanceDataset(val_df_c, processor, CFG_C.val_image_dir, CFG_C.image_size, CFG_C.max_token_len)
test_dataset_c = StanceDataset(test_df_c, processor, CFG_C.test_image_dir, CFG_C.image_size, CFG_C.max_token_len, is_test=True)

train_loader_c = DataLoader(train_dataset_c, batch_size=CFG_C.batch_size, shuffle=True, num_workers=CFG_C.num_workers)
val_loader_c = DataLoader(val_dataset_c, batch_size=CFG_C.batch_size, shuffle=False, num_workers=CFG_C.num_workers)
test_loader_c = DataLoader(test_dataset_c, batch_size=CFG_C.batch_size, shuffle=False, num_workers=CFG_C.num_workers)

print(f"DataLoaders created.")
print(f"Training batches:   {len(train_loader_c)}")
print(f"Validation batches: {len(val_loader_c)}")
print(f"Test batches:       {len(test_loader_c)}")

gc.collect()

# **Main Model Architecture**

In [ ]:
#Defining the Model Architecture
class StanceClassifier(nn.Module):
    def __init__(self, model_name, num_classes):
        super().__init__()
        self.clip = CLIPModel.from_pretrained(model_name)
        projection_dim = self.clip.projection_dim

        # A deeper, 3-layer classifier for more capacity.
        self.classifier = nn.Sequential(
            nn.Linear(2 * projection_dim, projection_dim * 2),
            nn.ReLU(),
            nn.Dropout(0.4), # Slightly increased dropout for the larger head
            nn.Linear(projection_dim * 2, projection_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(projection_dim, num_classes)
        )

    def forward(self, input_ids, attention_mask, pixel_values):
        outputs = self.clip(
            input_ids=input_ids,
            attention_mask=attention_mask,
            pixel_values=pixel_values
        )
        image_features = outputs.image_embeds
        text_features = outputs.text_embeds
        combined_features = torch.cat((image_features, text_features), dim=1)
        logits = self.classifier(combined_features)
        return logits

# Instantiate the Model

In [ ]:
model_c = StanceClassifier(
    model_name=CFG_C.model_name,
    num_classes=CFG_C.num_classes
).to(CFG_C.device)
print(f"Model instantiated.")

# **Defining the Training Engine with COSINE Scheduler**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = AdamW([
    {'params': model_c.clip.parameters(), 'lr': CFG_C.learning_rate_base},
    {'params': model_c.classifier.parameters(), 'lr': CFG_C.learning_rate_head}
])
num_training_steps = CFG_C.epochs * len(train_loader_c)

lr_scheduler = get_scheduler(
    name="cosine", # Using the more advanced cosine scheduler
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps
)
print("Optimizer and COSINE LR Scheduler are ready.")

# Helper Functions

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, scheduler, device):
    model.train()
    total_loss = 0
    progress_bar = tqdm(loader, desc="Training", leave=False)
    for batch in progress_bar:
        pixel_values = batch['pixel_values'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids, attention_mask, pixel_values)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=f"{loss.item():.4f}")
    return total_loss / len(loader)

def validate_one_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        progress_bar = tqdm(loader, desc="Validating", leave=False)
        for batch in progress_bar:
            pixel_values = batch['pixel_values'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask, pixel_values)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    avg_loss = total_loss / len(loader)
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro', zero_division=0)
    return avg_loss, accuracy, precision, recall, f1

print("Helper functions are ready.")

# **The Main Training Loop**

In [ ]:
history_c = {
    'train_loss': [], 'val_loss': [], 'val_accuracy': [],
    'val_precision': [], 'val_recall': [], 'val_f1': []
}
best_val_f1 = 0.0
model_path_c = os.path.join(CFG_C.output_dir, 'best_model_subtask_c_advanced.pth')

print("\n Starting Training for Subtask C: ")
for epoch in range(CFG_C.epochs):
    print(f"\n===== Epoch {epoch + 1}/{CFG_C.epochs} =====")
    train_loss = train_one_epoch(model_c, train_loader_c, optimizer, criterion, lr_scheduler, CFG_C.device)
    val_loss, val_acc, val_prec, val_rec, val_f1 = validate_one_epoch(model_c, val_loader_c, criterion, CFG_C.device)
    history_c['train_loss'].append(train_loss); history_c['val_loss'].append(val_loss)
    history_c['val_accuracy'].append(val_acc); history_c['val_precision'].append(val_prec)
    history_c['val_recall'].append(val_rec); history_c['val_f1'].append(val_f1)

    print(f"Epoch {epoch + 1} Summary:")
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    print(f"Accuracy: {val_acc:.4f}, F1-Score (Macro): {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        print(f"New best F1-score! Saving model to {model_path_c}")
        torch.save(model_c.state_dict(), model_path_c)
    else:
        print("F1-score did not improve.")

print("\n Training Finished ")
print(f"Best validation F1-Score for Subtask C achieved: {best_val_f1:.4f}")

# **Prediction on Test Dataset**

In [ ]:
# Define the Prediction Function
def predict_subtask_c(model_path, test_loader, device):
    print("--> Instantiating model architecture for prediction...")
    model = StanceClassifier(
        model_name=CFG_C.model_name,
        num_classes=CFG_C.num_classes
    ).to(device)

    print(f"--> Loading best model weights onto device: {device}")
    model.load_state_dict(torch.load(model_path, map_location=device))

    model.eval()
    all_preds = []

    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc="Predicting on Test Set")
        for batch in progress_bar:
            pixel_values = batch['pixel_values'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask, pixel_values)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())

    return all_preds

# Run the Prediction and Create the Submission File
model_path = os.path.join(CFG_C.output_dir, 'best_model_subtask_c_advanced.pth')
predictions = predict_subtask_c(model_path, test_loader_c, CFG_C.device)

indices = test_df_c['index'].tolist()

print("\n Creating JSON Lines submission file: ")
submission_path = os.path.join(CFG_C.output_dir, 'submission.json')

with open(submission_path, 'w') as f:
    for index, prediction in zip(indices, predictions):
        result = {
            "index": index,
            "prediction": int(prediction)
        }
        f.write(json.dumps(result) + '\n')

print(f"\nSubmission file for Subtask C created successfully at: {submission_path}")